In [1]:
#inibizi hanes  s/v = s * (1/Vmax) + Km/Vmax



In [1]:
import pandas as pd
#mettere .xlsx dopo il nome del documento excel
df = pd.read_excel (r'C:\Users\thomas\Documents\università\valetti\2E.xlsx') 
df.columns = ['s', 'v_norm', 'v_inib', 'conc_inib'] 
msus= input("Enter the measure unit of substrate:")
msuv= input("Enter the measure unit of velocity :")
msui=input("Enter the measure unit of inhibent if present:")
print (df)
print('substrate:'+msus)
print('velocity:'+msuv)
print('inhibent concentration:'+msui)

Enter the measure unit of substrate:uM
Enter the measure unit of velocity :uM/s
Enter the measure unit of inhibent if present:uM
       s   v_norm   v_inib  conc_inib
0  2.000  0.02850  0.01100        1.0
1  1.000  0.02450  0.00900        NaN
2  0.600  0.02025  0.00600        NaN
3  0.400  0.01650  0.00450        NaN
4  0.267  0.01325  0.00296        NaN
5  0.200  0.01100  0.00210        NaN
substrate:uM
velocity:uM/s
inhibent concentration:uM


In [2]:
from bokeh.plotting import figure, show, output_notebook
param = dict(width = 700, height = 250,
             tools = 'wheel_zoom, reset,pan, box_zoom',
             tooltips = [( 'substrate',   '@s'), ( 'velocity',   '@v')],
            )
output_notebook()

Loading BokehJS ...

In [3]:
p = figure(x_axis_label='Concentrazione substrato '+msus,
           y_axis_label='Velocità di reazione '+ msuv,
           **param,
           )
p.circle( 's', 'v_norm', source=df )
p.square('s', 'v_inib', source=df) 
show( p )
print('the square values are the inhibited ones')

the square values are the inhibited ones


In [4]:
df['s/v_norm'] = df['s']/df['v_norm']
df['s/v_inib']= df['s']/df['v_inib']
df


,s,v_norm,v_inib,conc_inib,s/v_norm,s/v_inib
0,2.000,0.02850,0.01100,1.0,70.175439,181.818182
1,1.000,0.02450,0.00900,NaN,40.816327,111.111111
2,0.600,0.02025,0.00600,NaN,29.629630,100.000000
3,0.400,0.01650,0.00450,NaN,24.242424,88.888889
4,0.267,0.01325,0.00296,NaN,20.150943,90.202703
5,0.200,0.01100,0.00210,NaN,18.181818,95.238095


In [5]:
from scipy.stats import linregress
slope_norm, intercept_norm, r_value, p_value, std_err = linregress( df['s'], df['s/v_norm'] )
slope_norm, intercept_norm   
'The slope non-inhibited is {}, and the intercept non-inibited is {}'.format(slope_norm, intercept_norm)

'The slope non-inhibited is 28.79013990098982, and the intercept non-inibited is 12.431837606580064'

In [6]:
from scipy.stats import linregress
slope_inib, intercept_inib, r_value, p_value, std_err = linregress( df['s'], df['s/v_inib'] )
'The slope inhibited is {}, and the intercept inibited is {}'.format(slope_inib, intercept_inib)

'The slope inhibited is 50.59857783326544, and the intercept inibited is 73.53918876296385'

In [7]:
from numpy import array 

In [14]:
#faccio un grafico con le 2 rette per capire il tipo di inibizione
p1 = figure(title = 'Linearizzazione di Hanes',
            x_axis_label= 's', 
            y_axis_label= 's/v',
            **param,
            )
p1.circle( 's', 's/v_norm', source=df )
xmax = df['s'].max() 
x = array( [-3, xmax] )       
y = x * slope_norm + intercept_norm 
p1.line( x, y, color='green')

p1.square( 's', 's/v_inib', source=df)
     
y = x * slope_inib + intercept_inib 
p1.line( x, y, color='red')
show( p1 )

In [10]:
#calcolo il punto di incontro delle rette, fondamentale per definire il tipo di inibizione
slope_x= slope_norm-slope_inib

intercept_x= -intercept_norm+intercept_inib
intersec_x=intercept_x/slope_x
intersec_x
intersec_y= slope_norm*intersec_x+intercept_norm
intersec_y

p1.triangle( intersec_x, intersec_y, color='black')

show( p1 )
print("The intersection of the 2 lines is at:(" +str(intersec_x)+ ','+str(intersec_y)+'), the black dot stands for the interception point of the 2 lines.')


The intersection of the 2 lines is at:(-2.8020049554281625,-68.23827706346349), the black dot stands for the interception point of the 2 lines.


In [ ]:
#linearizzazione hanes


# s/v = s * (1/Vmax) + Km/Vmax
#equazione di una retta y=mx+n

#Dove 	s/v = y
#	s = x
#	B *(Km/Vmax) = n   
#	A/Vmax = m

#Consideriamo
#m1: m per la reazione senza inibitore
#n1: n per la reazione senza inibitore
#m2: m per la reazione con inibitore
#n2: n per la reazione con inibitore

#inibitore competitivo: m1=m2 perchè A=1
#B = 1 + i/Kei = n2/n1

#inibitore acompetitivo: n1=n2 perchè B=1
#A= 1+ i/Keis = m2/m1


In [12]:
A=slope_inib/slope_norm
B=intercept_inib/intercept_norm
Kei=df['conc_inib']/(B-1)
Keis=df['conc_inib']/(A-1)
Ki=1/(B-1)
compet='The Kei is: '+str(Kei)+' '+msus+', and the coefficient B is: '+str(B)

a_compet='The Keis is: '+str(Keis)+' '+msus+', and the coefficient A is: '+str(A)
non_compet='The Ki is: '+str(Ki)+' '+msus+ ', and the coefficents A and B are repectively :'+str(A)+str(B)

In [13]:
inib_type = input("According to the graph, which kind inibition do you want to study? competitive, non competitive, acompetitive:")

if inib_type == 'competitive':
    print(compet)
elif inib_type== 'non competitive':
    print(non_compet)
elif inib_type== 'acompetitive':
    print(a_compet)

According to the graph, which kind inibition do you want to study? competitive, non competitive, acompetitive:competitive
The Kei is: 0    0.203443
1         NaN
2         NaN
3         NaN
4         NaN
5         NaN
Name: conc_inib, dtype: float64 uM, and the coefficient B is: 5.915391681438968
